In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db
from pandasql import sqldf


In [3]:

sqladb = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",sep=",",header=None )
sqladb.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
sqladb.columns = ["age","workclass","fnlwgt","education","education_num","marital_status",
"occupation","relationship","race","sex","capital_gain","capital_loss",
"hours_per_week","native_country",'class']
sqladb.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
# 3 
#frequency count of education
fedu =  pd.value_counts(sqladb.education).to_frame().reset_index()
fedu.columns = ["Education", "Count"]
fedu

,Education,Count
0,HS-grad,10501
1,Some-college,7291
2,Bachelors,5355
3,Masters,1723
4,Assoc-voc,1382
5,11th,1175
6,Assoc-acdm,1067
7,10th,933
8,7th-8th,646
9,Prof-school,576


In [7]:
# frequency table of Occupation
focc =  pd.value_counts(sqladb.occupation).to_frame().reset_index()
focc.columns = ["occupation", "Count"]
focc

,occupation,Count
0,Prof-specialty,4140
1,Craft-repair,4099
2,Exec-managerial,4066
3,Adm-clerical,3770
4,Sales,3650
5,Other-service,3295
6,Machine-op-inspct,2002
7,?,1843
8,Transport-moving,1597
9,Handlers-cleaners,1370


In [8]:
# frequency table of relationship
frel =  pd.value_counts(sqladb.relationship).to_frame().reset_index()
frel.columns = ["occupation", "Count"]
frel

,occupation,Count
0,Husband,13193
1,Not-in-family,8305
2,Own-child,5068
3,Unmarried,3446
4,Wife,1568
5,Other-relative,981


In [10]:
# 4
pysqldf = lambda q: sqldf(q, globals())
q = """SELECT
            *
           FROM
              sqladb
           where 
              marital_status = "Married-civ-spouse" and education = "Masters" and workclass = "Private";
           """

pysqldf(q)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class


In [11]:
#5
q = """
      select
        workclass
        , avg(age)
        , min(age)
        , max(age)
      from
        sqladb
      group by
        workclass;
        
"""

In [12]:
pysqldf(q)

,workclass,avg(age),min(age),max(age)
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


In [13]:
#6
#age distribution

sqladb.groupby('native_country')['age'].mean()

native_country
 ?                             38.725557
 Cambodia                      37.789474
 Canada                        42.545455
 China                         42.533333
 Columbia                      39.711864
 Cuba                          45.768421
 Dominican-Republic            37.728571
 Ecuador                       36.642857
 El-Salvador                   34.132075
 England                       41.155556
 France                        38.965517
 Germany                       39.255474
 Greece                        46.206897
 Guatemala                     32.421875
 Haiti                         38.272727
 Holand-Netherlands            32.000000
 Honduras                      33.846154
 Hong                          33.650000
 Hungary                       49.384615
 India                         38.090000
 Iran                          39.418605
 Ireland                       36.458333
 Italy                         46.424658
 Jamaica                       35.592593
 

In [15]:
# 7
# Capital net gain column

sqladb[" Net-Capital-Gain" ] = sqladb["capital_gain"]- sqladb["capital_loss"] 

In [16]:
sqladb.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Net-Capital-Gain
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,2174
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
